<center><img src="img/logo_hse_black.jpg"></center>

<h1><center>Методы машинного обучения</center></h1>
<h2><center>Введение в NLP</center></h2>

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,8)

# Работа с текстом

Рассмотрим коллекцию новостных сообщений за первую половину 2017 года. Про каждое новостное сообщение известны:
* его заголовок и текст
* дата его публикации
* событие, о котором это новостное сообщение написано 
* его рубрика 

In [ ]:
!wget https://www.dropbox.com/s/m9byg5j1ngdha9w/news.csv?dl=0 -O ./news.csv

In [ ]:
df = pd.read_csv('./news.csv', encoding='utf8')
df.head()

In [ ]:
print(df.text[0])

In [ ]:
df.shape

In [ ]:
df.loc[:, 'class'].value_counts()

In [ ]:
df.event.value_counts()

### Токенизация

* **Токенизация** - это процесс разбиения текста на элементарные состовляющие (токены)
* Токенами обычно являются слова или предложения. Для обобщения, также всключим сюда символы.


* "Кролики - это не только ценный мех, но и 3-4 килограмма диетического, легкоусвояемого мяса"
* "Через 1.5 часа поеду в Гусь-Хрустальный."

In [ ]:
import nltk
from nltk.tokenize import RegexpTokenizer, wordpunct_tokenize, word_tokenize

* Обычно все ограничевается использованием специальных **регулярных** выражений, разработанных на все случаи жизни

In [ ]:
line = u"Через 1.5 часа поеду в Гусь-Хрустальный."
tokenizer = RegexpTokenizer('\w+| \$ [\d \.]+ | S\+')
for w in tokenizer.tokenize(line):
    print(w)

In [ ]:
line = u"Через 1.5 часа поеду в Гусь-Хрустальный."
for w in wordpunct_tokenize(line):
    print(w)

In [ ]:
line = u"Через 1.5 часа поеду в Гусь-Хрустальный."
for w in word_tokenize(line):
    print(w)

In [ ]:
def words_only(text):
    return " ".join(word_tokenize(text))


df.text = df.text.str.lower()
df.loc[:, 'text'] = df.text.apply(words_only)

Результат:

In [ ]:
print(df.text.iloc[0])

In [ ]:
# Еще можно дополинительно оставить только русские слова

import re
regex = re.compile(u"[А-Яа-я]+")

def words_only(text, regex=regex):
    return " ".join(regex.findall(text))


df.loc[:, 'text'] = df.text.apply(words_only)

Результат:

In [ ]:
print(df.text.iloc[0])

### Самые частые слова

In [ ]:
from nltk import FreqDist
n_types = []
n_tokens = []
tokens = []
fd = FreqDist()

for index, row in df.iterrows():
    tokens = row['text'].split()
    fd.update(tokens)
    n_types.append(len(fd))
    n_tokens.append(sum(fd.values()))
    
for i in fd.most_common(10):
    print(u'{}: {}'.format(i[0], i[1]))

## Обработка текстов



### Удаление стоп-слов

Для удаления стоп-слов воспользуемся готовым словарем из nltk

In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
mystopwords = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', u'также',  'т', 'д', '-', '-']

print(mystopwords)
def  remove_stopwords(text, mystopwords = mystopwords):
    try:
        return u" ".join([token for token in text.split() if not token in mystopwords])
    except:
        return u""
    
df.text = df.text.apply(remove_stopwords)   

In [ ]:
df.text[0]

### Лемматизация

Для русского языка есть 2 неплохих лемматизатора - pymorphy и mystem

In [ ]:
!pip install pymystem3

In [ ]:
!pip install pymorphy2
!pip install -U pymorphy2-dicts-ru
!pip install -U pymorphy2-dicts-uk

In [ ]:
import pymorphy2

In [ ]:
m = pymorphy2.MorphAnalyzer()
m.normal_forms('митинги')

In [ ]:
m.normal_forms('использовался')

Pymorphy умеет только по одному слову

In [ ]:

def lemmatize(text, mystem=m):
    try:
        return " ".join(m.normal_forms(word)[0] for word in text.split(' '))
    except:
        return " "
    
df.loc[:, 'text_morphy'] = df.text.apply(lemmatize)


In [ ]:
df.loc[0, 'text_morphy']

In [ ]:
from pymystem3 import Mystem

m = Mystem()

In [ ]:
m.lemmatize('митинги прошли')

Mystem может сразу предложениями

In [ ]:
%%time 

def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "

df.text = df.text.apply(lemmatize)

In [ ]:
df.text[0]

### Удаление стоп-лемм

In [ ]:
mystoplemmas = [u'который', u'прошлый', u'сей', u'свой', u'наш', u'мочь']
def  remove_stoplemmas(text, mystoplemmas = mystoplemmas):
    try:
        return " ".join([token for token in text.split() if not token in mystoplemmas])
    except:
        return ""

df.text = df.text.apply(remove_stoplemmas)  

In [ ]:
df.text.head()

Самые частые леммы:

In [ ]:
lemmata = []
for index, row in df.iterrows():
    lemmata += row['text'].split()
fd = FreqDist(lemmata)
for i in fd.most_common(10):
    print(u'{}: {}'.format(i[0], i[1]))

## Извлечение частых словосочетаний

## N-граммы (n-grams)


Можно найти все возможные словосочетания (n-gramm\`ы) в тексте, а можно отдельно выделять часто встречаемые, для этого можно считать разные меры "связанности" двух соседних слов
* $w_1$, $w_2$ - слова
* $p(w_1)$, $p(w_2)$ - частоты слов
* $p(w_1, w_2)$ - частота биграммы


Например
* $PMI(w_1, w_2) = \log\frac{p(w_1, w_2)}{p(w_1)p(w_2)}$
* $\text{T-score}(w_1, w_2) = \frac{p(w_1,w_2) - p(w_1)p(w_2)}{p(w_1, w_2)/N}$
* ...

Переезжаем из DataFrame в списки:

In [ ]:
tokens_by_topic = []
for event in df.event.unique():
    tokens = []
    sample = df[df.event==event]
    for i in range(len(sample)):
        tokens += sample.text.iloc[i].split()
    tokens_by_topic.append(tokens)

Выберем событие, из текстов про которое будем извлекать ключевые слова:

In [ ]:
event_id = 3

Извлекаем биграммы по разным мерам связности:

In [ ]:
%%time 
import nltk
from nltk.collocations import *
N_best = 100 # число извлекаемых биграм

bigram_measures = nltk.collocations.BigramAssocMeasures() # класс для мер ассоциации биграм
finder = BigramCollocationFinder.from_words(tokens_by_topic[event_id]) # класс для хранения и извлечения биграм
finder.apply_freq_filter(3) # избавимся от биграм, которые встречаются реже трех раз
raw_freq_ranking = [' '.join(i) for i in finder.nbest(bigram_measures.raw_freq, N_best)] # выбираем топ-10 биграм по частоте 
tscore_ranking = [' '.join(i) for i in finder.nbest(bigram_measures.student_t, N_best)] # выбираем топ-10 биграм по каждой мере 
pmi_ranking =  [' '.join(i) for i in finder.nbest(bigram_measures.pmi, N_best)]
chi2_ranking =  [' '.join(i) for i in finder.nbest(bigram_measures.chi_sq, N_best)]

Результаты:

In [ ]:
rankings = pd.DataFrame({ 'chi2': chi2_ranking, 't-score' : tscore_ranking, 'pmi': pmi_ranking, 'raw_freq':raw_freq_ranking})
rankings = rankings[['raw_freq', 'pmi', 't-score', 'chi2', ]]
rankings.head(10)

In [ ]:
# И в дальнейшем такие частые словосочетания рассматривать как одно слово, например таким образом:
'дмитрий_медведев'

## Вычисление сходства

С помощью `TfidfVectorizer` и `pairwise_distances` расчитайте косинусное расстояние между всеми парами документов к корпусе

Запишите результат в переменную `S`

In [ ]:
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances

In [ ]:
vect = TfidfVectorizer() # векторизатор для получения мешка слов с tf-idf

In [ ]:
texts = df.text.values
X = vect.fit_transform(texts)

In [ ]:
vocab = vect.get_features() # cловарь
len(texts)
len(vocab)

In [ ]:
X # матрица с мешком слов

In [ ]:
sims = pairwise_distances(X, metric='cosine')

In [ ]:
sims.shape

In [ ]:
plt.figure(figsize = (10,10))
sns.heatmap(data=sims, cmap = 'Spectral').set(xticklabels=[],yticklabels=[])
